In [1]:
import mne
import pandas as pd
import numpy as np
import os

base_path = "/Users/sziwia/Documents/projekty health/seizure-risk-analysis/ds005873/sub-001/ses-01"
run = "07"

raw_mov = mne.io.read_raw_edf(f"{base_path}/mov/sub-001_ses-01_task-szMonitoring_run-{run}_mov.edf", preload=True, verbose=False)
events = pd.read_csv(f"{base_path}/eeg/sub-001_ses-01_task-szMonitoring_run-{run}_events.tsv", sep='\t')

sz_row = events[events['eventType'].str.contains('sz', na=False)].iloc[0]
sz_start, sz_end = sz_row['onset'], sz_row['onset'] + sz_row['duration']
sfreq = raw_mov.info['sfreq']

In [2]:
dataset = []
times = np.arange(0, raw_mov.times[-1] - 10, 5)

for t in times:
    t_end = t + 10
    acc_win = raw_mov.copy().crop(tmin=t, tmax=t_end).get_data()
    mag = np.sqrt(np.sum(acc_win**2, axis=0))
    
    dataset.append({
        'acc_mean': np.mean(mag),
        'acc_std': np.std(mag),
        'acc_max': np.max(mag),
        'target': 1 if (t < sz_end and t_end > sz_start) else 0
    })

df_final = pd.DataFrame(dataset)

In [3]:
out_dir = "/Users/sziwia/Documents/projekty health/seizure-risk-analysis/data/processed"
os.makedirs(out_dir, exist_ok=True)
df_final.to_csv(f"{out_dir}/features.csv", index=False)
print("saved")

saved
